# 3D Discharge Profiel Boundary Conditions

Boundary condition files have the most insane file format of all the insane Delft3D-FLOW file formats! Therefore, making input boundary conditions warrant a separate notebook.

In [2]:
import sys
sys.path.append('../')
from pyd3d.utils import formatSci

In [3]:
# from pyd3d.input.Discharge3Dprofile

In [40]:
import os
import numpy as np
from pyd3d.utils import formatSci
from pyd3d.input.mdf import Mdf
from pyd3d.input.sedmor import Sed
from IPython.display import Markdown as md
import pandas as pd
from decimal import Decimal

In [41]:
folderpath = "/Users/julesblom/pyDelft3D-FLOW/example_input/Run01"
model = D3DModel(folderpath)
filenames = model.filenames

## Read some values from .mdf

In [43]:
mdf = Mdf(filenames['mdf'])
timestep = float(mdf.data['Dt'][0])
timestep

Readding /Users/julesblom/pyDelft3D-FLOW/example_input/Run01/36km_200m_W60Channel.mdf


0.3

In [69]:
sigma_layer_percentages = mdf.data['Thick']
nr_sigma_layers = int(mdf.data['MNKmax'][2])

## Get some stuff from .sed

In [46]:
sediments = Sed(filenames['sed'])
sediments.sediment_names

['Sediment vfsand', 'Sediment msilt']

In [48]:
density_sed = float(sediments.data['Sediment0']['RhoSol']) # [kg/m3]
density_sed

2650.0

## User defined values

In [77]:
# should be read from depth ideally
init_depth = 300   # [m] at bc location # ureg.meter 

sigma_layer_fractions = np.divide(sigma_layer_percentages, 100)

np.multiply(sigma_layer_fractions, init_depth)

array([21.54, 18.  , 16.5 , 15.  , 13.5 , 12.  , 11.4 , 10.8 , 10.2 ,
        9.6 ,  9.  ,  8.7 ,  8.4 ,  8.1 ,  7.8 ,  7.5 ,  7.2 ,  6.9 ,
        6.6 ,  6.3 ,  6.  ,  5.7 ,  5.4 ,  5.1 ,  4.8 ,  4.5 ,  4.2 ,
        3.9 ,  3.6 ,  3.3 ,  3.  ,  2.85,  2.7 ,  2.55,  2.4 ,  2.25,
        2.1 ,  1.95,  1.8 ,  1.65,  1.5 ,  1.35,  1.2 ,  1.05,  0.9 ,
        0.81,  0.69,  0.6 ,  0.45,  0.39,  0.36,  0.33,  0.3 ,  0.3 ,
        0.27,  0.27,  0.27,  0.27,  0.27,  0.24,  0.24,  0.24,  0.24,
        0.21,  0.21,  0.21,  0.18,  0.18,  0.18,  0.18,  0.18,  0.15,
        0.15,  0.15,  0.15,  0.12,  0.12,  0.12,  0.09,  0.09])

In [49]:
discharge_total = 4500 # [m3/s]

sand_dens = 19.75 # kg/m3
silt_dens = 19.75 # kg/m3

nr_discharge_layers = 53 

sediments = [
    {
        'name': 'sand',
        "mass_conc": sand_mass_conc
    },
    {
        'name': 'silt',
        "mass_conc": silt_mass_conc
    }]

start_discharge_time = 9 # [min]
discharge_duration = 39 # [min]

In [50]:
input_dict = dict(timestep = timestep,
                  nr_sigma_layers=nr_sigma_layers,
                  start_discharge_time=start_discharge_time,
                  discharge_duration=discharge_duration,
                  nr_discharge_layers=nr_discharge_layers
                 )

In [53]:
def isTimestepMultiple(value):
    return value % timestep < 1e-9

In [65]:
# inflow_height = 


array([   41.78272981,    50.        ,    54.54545455,    60.        ,
          66.66666667,    75.        ,    78.94736842,    83.33333333,
          88.23529412,    93.75      ,   100.        ,   103.44827586,
         107.14285714,   111.11111111,   115.38461538,   120.        ,
         125.        ,   130.43478261,   136.36363636,   142.85714286,
         150.        ,   157.89473684,   166.66666667,   176.47058824,
         187.5       ,   200.        ,   214.28571429,   230.76923077,
         250.        ,   272.72727273,   300.        ,   315.78947368,
         333.33333333,   352.94117647,   375.        ,   400.        ,
         428.57142857,   461.53846154,   500.        ,   545.45454545,
         600.        ,   666.66666667,   750.        ,   857.14285714,
        1000.        ,  1111.11111111,  1304.34782609,  1500.        ,
        2000.        ,  2307.69230769,  2500.        ,  2727.27272727,
        3000.        ,  3000.        ,  3333.33333333,  3333.33333333,
      

In [67]:
np.divide(discharge_layer_sigma_percentages, 100)

NameError: name 'discharge_layer_sigma_percentages' is not defined

In [37]:
isTimestepMultiple(start_discharge_time)

True

In [39]:
isTimestepMultiple(discharge_duration)

True

In [51]:
def allZeroesRecords(nr_sigma_layers=80):
    all_zeros_line = '  '.join('0.0000000e+000' for i in range(nr_sigma_layers))
    line_all_zero = f"{all_zeros_line}  {all_zeros_line}" # left + right
    
    return line_all_zero

In [52]:
def makeBcTimes(timestep=0.3, start_discharge_time=15, discharge_duration=45):
    # Discharge is linearly interpolated between timesteps
    # Therefore, we need to add one timestep before the start and one timestep after in which discharges are zero
    end_discharge_time = start_discharge_time + discharge_duration # [min]
    pre_start_discharge_time = start_discharge_time - timestep  # [min]
    post_end_discharge_time = end_discharge_time + timestep  # [min]    

    times = [0, 
             pre_start_discharge_time, 
             start_discharge_time,
             end_discharge_time, 
             post_end_discharge_time, 
             end_time
            ]
    
    formatted_times = [formatSci(time) for time in times]
    
    return formatted_times

In [56]:
def calcDischargePerCell(discharge_total=None, sigma_layer_percentages=[], nr_zero_discharge_layers=None):
    if not discharge_total:
        raise Exception("No discharge_total given!")
    if not nr_zero_discharge_layers:
        raise Exception("No nr_zero_discharge_layers given!")
    
    discharge_layer_sigma_percentages = sigma_layer_percentages[nr_zero_discharge_layers:] # select only the discharge parts
    discharge_layers_sigma_percentage_sum = np.array(discharge_layer_sigma_percentages).sum() # sum it

    print(f"Now divide {discharge_total} $m^3/s$ discharge among the bottom {discharge_layers_sigma_percentage_sum}% height of sigma layers")
    discharge_records_list = discharge_layer_sigma_percentages/discharge_layers_sigma_percentage_sum * discharge_total
    
    return discharge_records_list

In [57]:
def makeDischargeRecords(discharge_bc_list=None, nr_sigma_layers=80, nr_discharge_layers = 53):
    '''
    Pass a list of float values for discharge layer 
    Returns a formatted string of records
    '''
    if not len(discharge_bc_list):
        raise Exception("discharge_records should be be a list")
    
    nr_zero_discharge_layers = nr_sigma_layers - nr_discharge_layers  # move this to class self props

    discharge_records = '  '.join(formatSci(discharge) for discharge in discharge_bc_list)
    
    print(f"The first {nr_zero_discharge_layers} 'cells' are all zeroes, then the next {nr_discharge_layers} layers contain the discharge values")
    
    zero_records_discharge_layer = '  '.join('0.0000000e+000' for i in range(nr_zero_discharge_layers))
    # first the cells where discharge is zero, then the cells that have values for discharge 
    discharge_line = f"{zero_records_discharge_layer}  {discharge_records}"

    # Each line has two discharge 'points' so repeat it twice
    complete_line_records = f'{discharge_line}  {discharge_line}' # A + B

    return complete_line_records

In [58]:
line_all_zero = allZeroesRecords(nr_sigma_layers)

In [59]:
def generateVerticalBC(filename='untitled', timestep = 0.3, nr_sigma_layers=80, start_discharge_time=0,
                       discharge_duration=0, nr_discharge_layers=53):
    if discharge_duration == 0:
        raise Exception("discharge_duration cannot be 0")
    
    if not Decimal(discharge_duration) % Decimal(timestep) < 10e-9:
        raise Exception("Discharge duration is not a multiple of timestep")

    times = makeBcTimes(timestep, start_discharge_time, discharge_duration)    
    line_all_zero = allZeroesRecords(nr_sigma_layers)
        
    # --- Write Concentration file (.bcc) ---
    # time + A + B
    bcc_filename = filename + ".bccrecords"
    with open(bcc_filename, 'w') as bcc_file:
        # why not use a f multiline string here f'''{ line below} '''
        # what if more than 6 records? this approach is inflexible
        # for each discharge write these six lines
            # for time in times:
            
        for sed in sediments:
            bcc_file.write(sed['name'] + '\n')
            conc_discharge_list = [sed['mass_conc'] for i in range(nr_discharge_layers)]

            line_with_conc_records = makeDischargeRecords(conc_discharge_list)

            for i, time in enumerate(times):
                if i == 2 or i == 3:
                    write_line =  f" {time}  {line_with_conc_records}\n"
                    bcc_file.write(write_line)
                else:
                    write_line = f" {time}  {line_all_zero}\n"
                    bcc_file.write(write_line)
        
    # discharges inflow of water (different per vertical gridcell)
    nr_zero_discharge_layer = nr_sigma_layers - nr_discharge_layers
    discharges_list = calcDischargePerCell(discharge_total, sigma_layer_percentages, nr_zero_discharge_layer)
    line_with_discharge_records = makeDischargeRecords(discharges_list, nr_sigma_layers, nr_discharge_layers)        
    # Format: time + A + B
    bct_filename = filename + ".bctrecords"
    with open(bct_filename, 'w') as bct_file:
        linesToWrite = [
            f" {times[0]}  {line_all_zero}\n",
            f" {times[1]}  {line_all_zero}\n",
            f" {times[2]}  {line_with_discharge_records}\n",
            f" {times[3]}  {line_with_discharge_records}\n",
            f" {times[4]}  {line_all_zero}\n",
            f" {times[5]}  {line_all_zero}\n"
        ]
        bct_file.writelines(line for line in linesToWrite)

# And now, magic!

In [ ]:
generateVerticalBC('test', **input_dict)